Extract, profiling and cleaning data
---

### Step 1: Extract data

1. connect to NYC Open Data with API Key
2. pull specific dataset as a pandas dataframe
3. Look at shape of extracted data

In [ ]:
pip install sodapy

In [1]:
# import libraries
import pandas as pd
from sodapy import Socrata

In [7]:
data_url = 'data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set = 'uip8-fykc'    # The data set at the API endpoint (311 data in this case)
app_token = 'Xon2wgccsDGbvkt07HB7X2qEO'   # The App Token code created in the prior steps

# full URL to look at data on NYC Open Data
# https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

In [8]:
# create the client that points to the API endpoint
client = Socrata(data_url, app_token, timeout = 200)

In [9]:
print(f"client name is: {client}")
print(f"client data type is: {type(client)}")

client name is: <sodapy.socrata.Socrata object at 0x7fc22065caf0>
client data type is: <class 'sodapy.socrata.Socrata'>


In [10]:
# test the connection to NYC Open Data

# retrieve the first 100 rows from the data_set
test_results = client.get(data_set, limit = 100)

# the test_results are returned as JSON object from the API
# the sodapy library converts this JSON object to a python list of dictionaries
# now, convert the list of dictionaries to a pandas data frame
test_results_df = pd.DataFrame.from_records(test_results)

In [11]:
# examine the test_results_df pandas dataframe
test_results_df.head(2)

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,x_coord_cd,y_coord_cd,latitude,longitude,geocoded_column,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_efsh_h5xi
0,238013474,2021-12-18T00:00:00.000,157,RAPE 1,104.0,RAPE,PL 1303501,F,Q,105,...,1057636,203954,40.72618713000002,-73.73523390399998,"{'type': 'Point', 'coordinates': [-73.73523390...",63,3,16,63,24337
1,236943583,2021-11-25T00:00:00.000,263,"ARSON 2,3,4",114.0,ARSON,PL 1501500,F,K,69,...,1014340,171320,40.63686750900007,-73.89158591899997,"{'type': 'Point', 'coordinates': [-73.89158591...",5,2,8,42,13827


In [17]:
# next, get the total number of records in our the entire data set
total_record_count = client.get(data_set, select = "COUNT(*)")
print(f"total records in {data_set}: {total_record_count}")

total records in uip8-fykc: [{'COUNT': '155507'}]


In [18]:
# next, get the total number of records in our target data set
target_record_count = client.get(data_set,
                                 where = "ARREST_DATE > '2000-01-01'",
                                 select= "COUNT(*)")
print(f"target records in {data_set}: {target_record_count}")

target records in uip8-fykc: [{'COUNT': '155507'}]


In [19]:
# loop through data set to pull all rows in chunks (cannot pull all rows at once)

# measure time this function takes
import time
start_time = time.time()

start = 0             # start at 0
chunk_size = 2000     # fetch 2000 rows at a time
results = []          # empty out our result list
record_count = target_record_count

while True:
    
    # fetch the set of records starting at 'start'
    results.extend(client.get(data_set,
                              where = "ARREST_DATE > '2000-01-01'",
                              offset = start,
                              limit = chunk_size))
    
    # update the starting record number
    start = start + chunk_size
    
    # if we have fetched all of the records (we have reached record_count), exit loop
    if (start > int(record_count[0]['COUNT'])):
        break
        
# convert the list into a pandas data frame
data = pd.DataFrame.from_records(results)

end_time = time.time()
print(f"loop to {round(end_time - start_time, 1)} seconds")

data.info()

loop to 75.4 seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155507 entries, 0 to 155506
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   arrest_key         155507 non-null  object
 1   arrest_date        155507 non-null  object
 2   pd_cd              155478 non-null  object
 3   pd_desc            155404 non-null  object
 4   ky_cd              155404 non-null  object
 5   ofns_desc          155404 non-null  object
 6   law_code           155507 non-null  object
 7   law_cat_cd         154114 non-null  object
 8   arrest_boro        155507 non-null  object
 9   arrest_precinct    155507 non-null  object
 10  jurisdiction_code  155507 non-null  object
 11  age_group          155507 non-null  object
 12  perp_sex           155507 non-null  object
 13  perp_race          155507 non-null  object
 14  x_coord_cd         155507 non-null  object
 15  y_coord_cd         155507 non-null  object
 16 

### Step 2: Data Profiling

1. Distinct values per column
2. Null values per column
3. Summary statistics per numeric column

In [20]:
# what are the columns in our dataframe?
data.columns

Index(['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc',
       'law_code', 'law_cat_cd', 'arrest_boro', 'arrest_precinct',
       'jurisdiction_code', 'age_group', 'perp_sex', 'perp_race', 'x_coord_cd',
       'y_coord_cd', 'latitude', 'longitude', 'geocoded_column'],
      dtype='object')

In [21]:
# create a dataframe to gather information about each column
data_profiling_df = pd.DataFrame(columns = ["column_name",
                                            "column_type",
                                            "unique_values",
                                            "duplicate_values",
                                            "null_values",
                                            "non_null_values",
                                            "percent_null"])

In [22]:
# loop through each column to add rows to the data_profiling_df dataframe
for column in data.columns:
    
    info_dict = {}
    
    try:
        info_dict["column_name"] = column
        info_dict["column_type"] = data[column].dtypes
        info_dict["unique_values"] = len(data[column].unique())
        info_dict["duplicate_values"] = (data[column].shape[0] - data[column].isna().sum()) - len(data[column].unique())
        info_dict["null_values"] = data[column].isna().sum()
        info_dict["non_null_values"] = data[column].shape[0] - data[column].isna().sum()
        info_dict["percent_null"] = round((data[column].isna().sum()) / (data[column].shape[0]), 3)
        
    except:
        print(f"unable to read column: {column}")
    
    data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
    
data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                              ascending = [False, False],
                              inplace=True)

unable to read column: geocoded_column


In [23]:
# lets look at the location column
data["geocoded_column"].head()

0    {'type': 'Point', 'coordinates': [-73.73523390...
1    {'type': 'Point', 'coordinates': [-73.89158591...
2    {'type': 'Point', 'coordinates': [-73.96041495...
3    {'type': 'Point', 'coordinates': [-73.91036538...
4    {'type': 'Point', 'coordinates': [-73.92439508...
Name: geocoded_column, dtype: object

In [24]:
type(data["geocoded_column"][0])

dict

In [25]:
# drop location column
data.drop(["geocoded_column"], axis = 1, inplace = True)

In [26]:
data_profiling_df

,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values,percent_null
0,arrest_key,object,155507,0,0,155507,0.000
16,latitude,object,34210,121297,0,155507,0.000
17,longitude,object,34210,121297,0,155507,0.000
15,y_coord_cd,object,29300,126207,0,155507,0.000
14,x_coord_cd,object,27466,128041,0,155507,0.000
6,law_code,object,1005,154502,0,155507,0.000
1,arrest_date,object,365,155142,0,155507,0.000
2,pd_cd,object,261,155217,29,155478,0.000
3,pd_desc,object,242,155162,103,155404,0.001
9,arrest_precinct,object,77,155430,0,155507,0.000


### Step 3: Data Cleansing

1. drop unneeded columns
2. drop duplicate rows
3. check for outliers

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155507 entries, 0 to 155506
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   arrest_key         155507 non-null  object
 1   arrest_date        155507 non-null  object
 2   pd_cd              155478 non-null  object
 3   pd_desc            155404 non-null  object
 4   ky_cd              155404 non-null  object
 5   ofns_desc          155404 non-null  object
 6   law_code           155507 non-null  object
 7   law_cat_cd         154114 non-null  object
 8   arrest_boro        155507 non-null  object
 9   arrest_precinct    155507 non-null  object
 10  jurisdiction_code  155507 non-null  object
 11  age_group          155507 non-null  object
 12  perp_sex           155507 non-null  object
 13  perp_race          155507 non-null  object
 14  x_coord_cd         155507 non-null  object
 15  y_coord_cd         155507 non-null  object
 16  latitude           1

In [28]:
# drop unneeded columns

drop_columns = ["pd_desc",
               "ky_cd",
               "ofns_desc",
               "x_coord_cd",
               "y_coord_cd"]

for column in drop_columns:
    try:
        data.drop(column, axis = 1, inplace = True)
    except:
        print(f"unable to drop {column}")

data.columns

Index(['arrest_key', 'arrest_date', 'pd_cd', 'law_code', 'law_cat_cd',
       'arrest_boro', 'arrest_precinct', 'jurisdiction_code', 'age_group',
       'perp_sex', 'perp_race', 'latitude', 'longitude'],
      dtype='object')

In [29]:
data[data.duplicated()]

,arrest_key,arrest_date,pd_cd,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,latitude,longitude


In [30]:
# find number of duplicate rows

print(f"number of duplicate rows: {len(data[data.duplicated()])}")

number of duplicate rows: 0


In [31]:
# drop duplicate rows

## drop duplicates here

## print new shape of data

In [32]:
len(data[data.duplicated()])

0